# Import packages

In [1]:
import sys
import os
sys.path.append('../../') # Add the root directory to sys.path

In [2]:
import pandas as pd
from stellarmapper.models.starnet import StarNet
import numpy as np

## Read data

In [3]:
'''
df_xp = pd.read_parquet('../../datasets/Gaia DR3/df_xp.parquet')
df_xp
'''

"\ndf_xp = pd.read_parquet('../../datasets/Gaia DR3/df_xp.parquet')\ndf_xp\n"

In [4]:
'''
data_dir = '../../datasets/Gaia DR3/XP'
files = [file for file in glob.glob(data_dir + '/*.vot')]

dict = {}
pattern = r'_(\d+)\.vot$'

for file in files:
    match = re.search(pattern, os.path.basename(file))
    numeric_id = match.group(1)
    df = parse_single_table(file).to_table().to_pandas()        
    dict[numeric_id] = df['flux'].values
''' 

"\ndata_dir = '../../datasets/Gaia DR3/XP'\nfiles = [file for file in glob.glob(data_dir + '/*.vot')]\n\ndict = {}\npattern = r'_(\\d+)\\.vot$'\n\nfor file in files:\n    match = re.search(pattern, os.path.basename(file))\n    numeric_id = match.group(1)\n    df = parse_single_table(file).to_table().to_pandas()        \n    dict[numeric_id] = df['flux'].values\n"

In [5]:
'''
df_xp.dropna(inplace=True)
df_xp.reset_index(inplace=True, drop=True)
df_xp
'''

'\ndf_xp.dropna(inplace=True)\ndf_xp.reset_index(inplace=True, drop=True)\ndf_xp\n'

In [6]:
df_xp = pd.read_parquet('./dataset.parquet')
df_xp

,source_id,teff_gspphot,logg_gspphot,mh_gspphot,spectraltype_esphs,Cat,flux
0,466475508969557888,18411.494141,3.5385,-0.3139,B,M,"[1.4043112e-15, 1.3886055e-15, 1.3119094e-15, ..."
1,3325513363920961408,16151.836914,3.7164,0.0395,B,M,"[1.4678052e-15, 1.3982165e-15, 1.3334084e-15, ..."
2,3375508943611375488,18823.841797,3.4316,0.2918,B,M,"[1.0764202e-15, 1.0605224e-15, 1.0162869e-15, ..."
3,4063973032805550592,15295.044922,3.4026,0.0999,G,M,"[9.53517e-16, 9.214246e-16, 8.3622285e-16, 7.9..."
4,454379128715344896,13149.229492,3.8305,-0.9505,B,LM,"[6.8294334e-16, 6.693247e-16, 6.1780524e-16, 5..."
...,...,...,...,...,...,...,...
17622,5912596947718581760,4950.860352,3.2933,-0.2100,K,LM,"[1.6002952e-17, 1.2129033e-17, 1.1508219e-17, ..."
17623,5912764520164418048,5681.474121,4.5653,-0.1923,G,LM,"[3.2488726e-17, 3.4947448e-17, 3.4823675e-17, ..."
17624,5912764520164428032,6002.563477,4.2886,-0.4792,G,LM,"[2.4685083e-17, 1.9302198e-17, 1.853873e-17, 1..."
17625,2263622213680760320,5369.893555,4.0945,-0.1173,K,LM,"[6.071518e-17, 5.3783307e-17, 5.0137515e-17, 4..."


In [7]:
#df_xp['Cat'].value_counts().plot(kind='bar')

## Classification

In [8]:
cols_to_be_dropped = ['source_id', 'teff_gspphot', 'logg_gspphot','mh_gspphot', 'spectraltype_esphs']
data = df_xp.drop(cols_to_be_dropped, axis=1, inplace=False)
data

,Cat,flux
0,M,"[1.4043112e-15, 1.3886055e-15, 1.3119094e-15, ..."
1,M,"[1.4678052e-15, 1.3982165e-15, 1.3334084e-15, ..."
2,M,"[1.0764202e-15, 1.0605224e-15, 1.0162869e-15, ..."
3,M,"[9.53517e-16, 9.214246e-16, 8.3622285e-16, 7.9..."
4,LM,"[6.8294334e-16, 6.693247e-16, 6.1780524e-16, 5..."
...,...,...
17622,LM,"[1.6002952e-17, 1.2129033e-17, 1.1508219e-17, ..."
17623,LM,"[3.2488726e-17, 3.4947448e-17, 3.4823675e-17, ..."
17624,LM,"[2.4685083e-17, 1.9302198e-17, 1.853873e-17, 1..."
17625,LM,"[6.071518e-17, 5.3783307e-17, 5.0137515e-17, 4..."


In [9]:
df_xp['Cat'].value_counts()

Cat
LM    11026
M      6601
Name: count, dtype: int64

In [10]:
from collections import Counter
from torch.utils.data import WeightedRandomSampler

counter = Counter(data['Cat'].to_numpy())

weight_map = {label: 1./count for label, count in counter.items()}

sampler = WeightedRandomSampler(weights=[weight_map[label] for label in data['Cat'].to_numpy()], num_samples=len(data), replacement=True)

In [11]:
'''
import matplotlib.pyplot as plt

# Customize the plot
plt.figure(figsize=(20, 7))

for i, row in df_xp.iterrows():
    if i > 30:
        break
    else:
        spec = row[:342]

        color = 'blue' if row['Cat'] == 'M' else 'orange'
        spec.plot(color=color, label=f'Row {i + 1} - Category: {row["Cat"]}')  

plt.title('Line Plot for the First 5 Rows')
plt.xlabel('Column Index')
plt.ylabel('Value')
plt.show()
'''

'\nimport matplotlib.pyplot as plt\n\n# Customize the plot\nplt.figure(figsize=(20, 7))\n\nfor i, row in df_xp.iterrows():\n    if i > 30:\n        break\n    else:\n        spec = row[:342]\n\n        color = \'blue\' if row[\'Cat\'] == \'M\' else \'orange\'\n        spec.plot(color=color, label=f\'Row {i + 1} - Category: {row["Cat"]}\')  \n\nplt.title(\'Line Plot for the First 5 Rows\')\nplt.xlabel(\'Column Index\')\nplt.ylabel(\'Value\')\nplt.show()\n'

In [12]:
# Set target column 1 for 'M' and 0 for 'LM'
data['Cat'] = data['Cat'].apply(lambda x: 1 if x == 'M' else 0)

In [13]:
X = np.vstack(data['flux'])
y = np.vstack(data['Cat'])

split = 0.8
train_size = int(len(data) * split)

X_train, X_test = np.split(X, [train_size])
y_train, y_test = np.split(y, [train_size])

In [ ]:
min = np.min(X)
max = np.max(X)

X_train = (X_train - min) / (max - min)

In [ ]:
num_labels = y_train.shape[1]
num_fluxes = X_train.shape[1]

print("Each spectrum contains", num_fluxes, "wavelength bins.")
print("Training set includes", X_train.shape[0], "spectra.")
print("Validation set includes", X_test.shape[0], "spectra.")

In [ ]:
model = StarNet(num_fluxes=num_fluxes, epochs = 10000, num_labels=num_labels, learning_rate=7e-6, prt_interval = 100, batch_size=32)
train_loss, val_loss = model.fit(X_train, y_train)

In [ ]:
save_dir = '../trained_models'
model.save_model(save_dir)

## Regression

In [ ]:
cols_to_be_dropped = ['source_id', 'spectraltype_esphs', 'Cat']
data = df_xp.drop(cols_to_be_dropped, axis=1, inplace=False)
data

In [ ]:
X = np.vstack(data['flux'])
y = data[['teff_gspphot', 'logg_gspphot', 'mh_gspphot']].to_numpy()

split = 0.8

train_size = int(len(data) * split)

X_train, X_test = np.split(X, [train_size])
y_train, y_test = np.split(y, [train_size])

In [ ]:
# normalize training labels before training
def normalize(labels):
    return (labels - mean) / std

mean = np.array([np.mean(y[:,0]), np.mean(y[:,1]), np.mean(y[:,2])])
std = np.array([np.std(y[:,0]), np.std(y[:,1]), np.std(y[:,2])])

y_train = normalize(y_train)
y_train

In [ ]:
min = np.min(X)
max = np.max(X)

X_train = (X_train - min) / (max - min)

In [ ]:
num_labels = y_train.shape[1]
num_fluxes = X_train.shape[1]

print("Each spectrum contains", num_fluxes, "wavelength bins.")
print("Training set includes", X_train.shape[0], "spectra.")
print("Validation set includes", X_test.shape[0], "spectra.")

In [ ]:
model = StarNet(num_fluxes=num_fluxes, epochs = 1000, num_labels=num_labels, learning_rate=7e-6, prt_interval = 100)
model.fit(X_train, y_train)